In [66]:
quran_file = 'id_text_with_orthographies.txt'

In [67]:
with open(quran_file) as fp:
    temp = fp.readlines()

In [68]:
verses = {}

for verse in temp:
    spls = verse.split('\t')
    verse_no = spls[0]
    verse_con = spls[1].strip()
    verses[verse_no] = verse_con

In [69]:
import json

with open('analysis.json') as fp:
    analysis_data = json.load(fp)

# with open('quran_prep.json') as fp:
#     verses = json.load(fp)

In [70]:
import csv

vavs = {}

with open('Vav.csv') as fp:
    reader = csv.reader(fp, delimiter=',')
    for i, row in enumerate(reader):
        if i == 0:
            continue
        else:
            vavs[row[0]] = int(row[1])
vavs_keys = set(vavs.keys())


In [50]:
vavs['والارض']

0

In [71]:
import re

verses_prep = {}
with_diacritic_verses_prep = {}
for verse_id, text in verses.items():
    text = re.sub(r'[\s]+', ' ', text)
    with_diacritic_verse_tokens = [w for w in re.split(r' \b',text)]
    text = re.sub(r'[' + analysis_data['pun'] + 'ّ]', '', text)
    for key, val in analysis_data['repl'].items():
        text = re.sub(r'[' + key + ']', val, text)
    verse_tokens = [w for w in text.split(' ') if w!='']
    
    commons = set.intersection(set(verse_tokens), vavs_keys)
    if commons:
        verse_tokens_new = []
        with_diacritic_verse_tokens_new=[]
        for i, w in enumerate(verse_tokens):
            command='SEPARATE'
            if w in commons:
                if vavs[w] == 0:
                    if w == 'وال':
                        if verse_tokens[i - 1] == 'من':
                            command='DONT_SEPARATE'
                    elif w == 'ورد':
                        if verse_tokens[i + 1] == 'ماء':
                            command='DONT_SEPARATE'
                    elif w == 'وكل':
                        if verse_tokens[i + 1] == 'بكم':
                            command='DONT_SEPARATE'
                    elif w == 'ولي':
                        if not (verse_tokens[i + 1] == 'فيها' or verse_tokens[i + 1] == 'نعجة' or verse_tokens[i + 1] == 'دين'):
                            command='DONT_SEPARATE'
                    elif w == 'وليتم':
                        if verse_tokens[i + 1] == 'مدبرين':
                            command='DONT_SEPARATE'
                    elif w == 'وليقولوا':
                        if verse_tokens[i + 1] == 'قولا':
                            command='DONT_SEPARATE'
                    elif w == 'وهب':
                        if verse_tokens[i - 1] == 'الذي':
                            command='DONT_SEPARATE'
                    elif w == 'وفي':
                        if verse_tokens[i - 1] == 'الذي':
                            command='DONT_SEPARATE'
                    elif w == 'والارض':
                        if i == 0 and verse_tokens[i + 1] in ['ذات', 'و']:
                            command='DONT_SEPARATE'
                    elif w == 'والسماء':
                        if i == 0 and verse_tokens[i + 1] in ['ذات', 'و']:
                            command='DONT_SEPARATE'
                    elif w == 'والشمس':
                        if i == 0 and verse_tokens[i + 1] == 'و':
                            command='DONT_SEPARATE'
                    elif w == 'والقران':
                        if verse_tokens[i + 1] in ['الحكيم', 'ذي', 'المجيد']:
                            command='DONT_SEPARATE'
                    elif w == 'والقمر':
                        if (i > 0 and verse_tokens[i - 1] == 'كلا') or (i == 0 and verse_tokens[i + 1] == 'اذا'):
                            command='DONT_SEPARATE'
                    elif w == 'والكتاب':
                        if verse_tokens[i + 1] == 'المبين':
                            command='DONT_SEPARATE'
                    elif w == 'والنهار':
                        if i == 0 and verse_tokens[i + 1] == 'اذا':
                            command='DONT_SEPARATE'
                    elif w == 'وهذا':
                        if i == 0 and verse_tokens[i + 1] == 'البلد':
                            command='DONT_SEPARATE'
                    elif w == 'وربك':
                        if i > 0 and verse_tokens[i - 1] == 'فلا':
                            command='DONT_SEPARATE'
                    elif w == 'والله':
                        if i > 0 and verse_tokens[i + 1] == 'ربنا':
                            command='DONT_SEPARATE'
                    elif w == 'وربنا':
                        if i > 0 and verse_tokens[i - 1] == 'بلي':
                            command='DONT_SEPARATE'
                    else:
                        command='DONT_SEPARATE'
                elif vavs[w] == 1:
                    command='DONT_SEPARATE'
            else:
                command='DONT_SEPARATE' 
            if command=='DONT_SEPARATE':
                verse_tokens_new.append(verse_tokens[i])
                with_diacritic_verse_tokens_new.append(with_diacritic_verse_tokens[i])
            else:
                with_diacritic_verse_tokens_new.append(str(with_diacritic_verse_tokens[i][:2]))
                with_diacritic_verse_tokens_new.append(with_diacritic_verse_tokens[i][2:])
                verse_tokens_new.append(str(verse_tokens[i][0]))
                verse_tokens_new.append(verse_tokens[i][1:])
        verses_prep[verse_id] = verse_tokens_new
        with_diacritic_verses_prep[verse_id] = with_diacritic_verse_tokens_new
    else:
        verses_prep[verse_id] = verse_tokens
        with_diacritic_verses_prep[verse_id] = with_diacritic_verse_tokens
quran_text = ' '.join([' '.join(val) for val in verses_prep.values()])
with_diacritic_quran_text = ' '.join([' '.join(val) for val in with_diacritic_verses_prep.values()])
words = re.split(r' \b',quran_text)
with_diacritic_words = re.split(r' \b',with_diacritic_quran_text)

In [83]:
words_set = sorted(set(words))
with_diacritic_words_set = sorted(set(with_diacritic_words))
words_set = ['[UNK]']+words_set
with_diacritic_words_set = ['[UNK]']+words_set

words_map = {w:i for i, w in enumerate(words_set)}
with open('word_codes.json', 'w') as fw:
    json.dump(words_map, fw, ensure_ascii=False)

with open('words_with_diacritics.txt', 'w') as fw:
    for w in with_diacritic_words_set:
        fw.write(w + '\n')

In [84]:
with open('quran_prep_with_diacritics.json', 'w') as fw:
    json.dump(with_diacritic_verses_prep, fw, ensure_ascii=False)

In [85]:
with open('quran_prep.json', 'w') as fw:
    json.dump(verses_prep, fw, ensure_ascii=False)

In [86]:
words_sub = set([w for w in words if w and w[0] == 'و'])
words_sub_sub = set([w for w in words_sub if w[1:] not in words])

In [87]:
with open('words_starts_with_v.txt', 'w') as fw:
    for w in words_sub:
        fw.write(w + '\n')

In [88]:
words_set = set([w for w in words if w[0] != 'و'])
words_one = [w for w in words_set if len(w) == 1]

In [89]:
index = {}
for verse_id, verse_seq in verses_prep.items():
    for word_id in [words_map[w] for w in verse_seq if w]:
        if not word_id in index:
            index[word_id] = set()
        index[word_id].add(verse_id)

In [90]:
for key in index:
    index[key] = sorted(list(index[key]))

with open('verse_index.json', 'w') as fw:
    json.dump(index, fw)

In [91]:
[(words[i - 3], words[i - 2], words[i - 1], words[i], words[i + 1]) for i in range(len(words)) if words[i] == 'والله']

[('الا', 'ان', 'قالوا', 'والله', 'ربنا')]